In [1]:
from datasets import load_dataset
from sentence_transformers.losses import CosineSimilarityLoss

from setfit import SetFitModel, SetFitTrainer

In [2]:
dataset = load_dataset("SetFit/SentEval-CR")

Found cached dataset json (/home/andres/.cache/huggingface/datasets/SetFit___json/SetFit--SentEval-CR-c9aa0d3185fe82a0/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
# Select N examples per class (8 in this case)
train_ds = dataset["train"].shuffle(seed=42).select(range(8 * 2))
test_ds = dataset["test"]

Loading cached shuffled indices for dataset at /home/andres/.cache/huggingface/datasets/SetFit___json/SetFit--SentEval-CR-c9aa0d3185fe82a0/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-80e075b3573a34a7.arrow


In [4]:
# Load SetFit model from Hub
model = SetFitModel.from_pretrained(
    "sentence-transformers/paraphrase-mpnet-base-v2",
    load_in_8bit=True
)

# Create trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    loss_class=CosineSimilarityLoss,
    batch_size=8,
    num_iterations=20, # Number of text pairs to generate for contrastive learning
    num_epochs=1 # Number of epochs to use for contrastive learning
)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [5]:
# Train and evaluate!
trainer.train()
metrics = trainer.evaluate()

***** Running training *****
  Num examples = 640
  Num epochs = 1
  Total optimization steps = 80
  Total train batch size = 8


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/80 [00:00<?, ?it/s]

***** Running evaluation *****


In [6]:
metrics

{'accuracy': 0.8764940239043825}

In [7]:
train_ds.num_rows

16

In [8]:
test_ds.num_rows

753